# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
print(len(df))
df.head()

8188


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [2]:
df.action.value_counts()
## In order to click you must view: so the number of viewers that also clicked
## are incorporated in the click number. 

view     6328
click    1860
Name: action, dtype: int64

In [5]:
## No one should be able to click without viewing.
## But we can check that by examining the unique ids
click_ids = set(df[df.action=='click']['id'].unique())
view_ids = set(df[df.action=='view']['id'].unique())
print("Number of viewers: {} \tNumber of clickers: {}".format(len(view_ids), len(click_ids)))
print("Number of Viewers who didn't click: {}".format(len(view_ids-click_ids)))
print("Number of Clickers who didn't view: {}".format(len(click_ids-view_ids)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


In [6]:
## Another quick test is to make sure there weren't people
## in both the experimental and control groups.
exp_ids = set(df[df.group=='experiment']['id'].unique())
control_ids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(exp_ids&control_ids)))


Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [8]:
## Create binary value to count instances within dataframe
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [12]:
df[df.group == 'control'].pivot(index = 'id', columns = 'action', values = 'count').fillna(value = 0)

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [13]:
## Subset data frames, and pivot action columns with binary values associated with user IDs as index
control_df = df[df.group == 'control'].pivot(index = 'id', columns = 'action', values = 'count').fillna(value = 0)
exp_df = df[df.group == 'experiment'].pivot(index = 'id', columns = 'action', values = 'count').fillna(value = 0)

In [14]:
print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control_df), len(exp_df)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control_df.click.sum(), exp_df.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control_df.click.mean(), exp_df.click.mean()))
control_df.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [19]:
## Use the control click rate calculated it earlier to extrapolate
## number of clicks expected based on the experimental group size
control_rate = control_df.click.mean()
expected_exp_clicks = control_rate * len(exp_df)
print(expected_exp_clicks)


838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [17]:
#Your code here
n = len(exp_df)
p = control_rate
variance = n * p * (1-p) ## from above
std = np.sqrt(variance)
print(std)

24.568547907005815


In [20]:
## Use standard deviation of control, actual experimental clicks, 
## and expected experimental clicks to calc z-score
actual_exp_clicks = exp_df.click.sum()
z_score = (actual_exp_clicks - expected_exp_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [21]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
## This p-value results in rejection of null hypothesis, whcih suggests
## the the new experimental page is more effective than the intial.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.